In [72]:
%pip install pandas
%pip install onnx

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [73]:
import torch
import torch.onnx
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import time
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(f"runs/training_experiment_{int(time.time())}")

import torch.nn.functional as F

In [74]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [75]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(X.min(), X.max())
    print(y)
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

tensor(0.) tensor(1.)
tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2,
        4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3])
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [76]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

class FashionCNN(nn.Module):
    def __init__(self):
        super(FashionCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.2, training=self.training)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, 0.2, training=self.training)
        x = self.fc3(x)
        return x

model = FashionCNN().to(device)
print(model)

Using mps device
FashionCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=3136, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)


In [77]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [78]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("runs/training_experiment")

def train(dataloader, model, loss_fn, optimizer, epoch):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)


        pred = model(X)
        loss = loss_fn(pred, y)


        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


        predicted = pred.argmax(dim=1)
        correct = (predicted == y).sum().item()
        accuracy = correct / y.size(0)


        step = epoch * len(dataloader) + batch
        writer.add_scalar("Loss/train", loss.item(), step)
        writer.add_scalar("Accuracy/train", accuracy, step)


        if batch % 100 == 0:
            loss_value, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss_value:>7f}  [{current:>5d}/{size:>5d}] accuracy: {accuracy:.4f}")

In [79]:
def test(dataloader, model, loss_fn, epochs):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            accuracy = correct / size
            writer.add_scalar("Accuracy/test", accuracy, epochs)

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [80]:
import torch.nn.functional as F

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer, t)
    test(test_dataloader, model, loss_fn, t)
writer.close()
print("Done!")

Epoch 1
-------------------------------
loss: 2.303984  [   64/60000] accuracy: 0.1250
loss: 0.331310  [ 6464/60000] accuracy: 0.8750
loss: 0.193615  [12864/60000] accuracy: 0.8906
loss: 0.192881  [19264/60000] accuracy: 0.9688
loss: 0.189661  [25664/60000] accuracy: 0.9219
loss: 0.092906  [32064/60000] accuracy: 0.9844
loss: 0.082677  [38464/60000] accuracy: 0.9688
loss: 0.236503  [44864/60000] accuracy: 0.9375
loss: 0.158102  [51264/60000] accuracy: 0.9844
loss: 0.138234  [57664/60000] accuracy: 0.9531
Test Error: 
 Accuracy: 97.4%, Avg loss: 0.085810 

Epoch 2
-------------------------------
loss: 0.054054  [   64/60000] accuracy: 0.9688
loss: 0.138822  [ 6464/60000] accuracy: 0.9688
loss: 0.022058  [12864/60000] accuracy: 1.0000
loss: 0.070928  [19264/60000] accuracy: 0.9844
loss: 0.113979  [25664/60000] accuracy: 0.9688
loss: 0.151819  [32064/60000] accuracy: 0.9531
loss: 0.077739  [38464/60000] accuracy: 0.9844
loss: 0.157055  [44864/60000] accuracy: 0.9688
loss: 0.174534  [51264

In [81]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [82]:
import torch

dummy_input = torch.randn(1, 1, 28, 28).to(device)

model.eval()

dummy_input = torch.randn(1, 1, 28, 28).to(device)

torch.onnx.export(
    model,
    dummy_input,
    "cnn.onnx",
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
    opset_version=11,
    export_params=True,
    do_constant_folding=True
)


In [83]:
model = FashionCNN().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [84]:
classes = [
    "0",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9",
]

model.eval()
x, y = test_data[2][0], test_data[2][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "1", Actual: "1"
